# ReverseNumber Example - two users

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

# Create two users

In [3]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [4]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  0254bae5a8eeeb19e9e1b7c32923414b9239233f55e532c86dd60ab4d86caa7d3e

~~~ serving get_instances request ~~~
Mounted ce236407-96b9-49cd-87e7-5106c241c40e at flows:reverse_number_atomic:mounts:03eaee4772a9ec3d137ac814d061e9234c61f08fe0d0f1b40868467fb5146956a9:ce236407-96b9-49cd-87e7-5106c241c40e
Fetched singleton ce236407-96b9-49cd-87e7-5106c241c40e

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: ce236407-96b9-49cd-87e7-5106c241c40e
message_paths: ['push_tasks:b7f60bc8-c51c-4ce5-9615-9f4c5ff9163b:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: ce236407-96b9-49cd-87e7-5106c241c40e
message_paths: ['push_tasks:19830c86-0f6d-40b4-a0df-f17d879986ab:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


In [5]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True


True

# USER 1

In [6]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  03eaee4772a9ec3d137ac814d061e9234c61f08fe0d0f1b40868467fb5146956a9

~~~ get_instances initiator ~~~
Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "ce236407-96b9-49cd-87e7-5106c241c40e",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "ce236407-96b9-49cd-87e7-5106c241c40e",
        "successful": true,
        "message": "Fetched local flow instance."
    }
}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 1b7d9373-b257-4e88-b080-89c55111dcfc
message_paths: ['push_tasks:f2011e59-8833-4608-bff8-ec707eb4cd98:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 1b7d9373-b257-4e88-b080-89c5511

In [7]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [8]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

Mounted 1b7d9373-b257-4e88-b080-89c55111dcfc at flows:reverse_number_sequential:mounts:local:1b7d9373-b257-4e88-b080-89c55111dcfc


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   mounts/
     03eaee4772a9ec3d137ac814d061e9234c61f08fe0d0f1b40868467fb5146956a9/
       ce236407-96b9-49cd-87e7-5106c241c40e/
         state
         init
         config_overrides
   default_dispatch_point
   parallel_dispatch
   init
   singleton
   flow_class_name

All flow instances:
 ce236407-96b9-49cd-87e7-5106c241c40e


In [11]:
status(cls[1])

 reverse_number_sequential/
   mounts/
     local/
       1b7d9373-b257-4e88-b080-89c55111dcfc/
         init
         state
         config_overrides
   parallel_dispatch
   flow_class_name
   default_dispatch_point
   singleton
   init

All flow instances:
 1b7d9373-b257-4e88-b080-89c55111dcfc
